In [1]:
# Import libraries and dependencies
import pandas as pd
from pathlib import Path
import hvplot.pandas
import panel as pn
import plotly.express as px
import seaborn as sns
%matplotlib inline

In [2]:
# Read in CSVs

# House Price Index dataframe (HPI)
hpi_csv_path = Path("Resources/HPI_PO_state.csv")
df_HPI_state = pd.read_csv(hpi_csv_path)

# Median home price by state dataframe
median_bystate_csv_path = Path("Resources/median_home_price_bystate.csv")
df_median_bystate = pd.read_csv(median_bystate_csv_path)

# Median home price by metro dataframe
median_bymetro_csv_path = Path("Resources/median_saleprice_uc_sfr_metro.csv")
df_median_bymetro = pd.read_csv(median_bymetro_csv_path)

# Latitude and Longitude by city dataframe
latlong_csv_path = Path("Resources/uscities.csv")
df_latlong = pd.read_csv(latlong_csv_path)

In [74]:
df_median_bymetro[df_median_bymetro['state'] == 'CA']

,year,state,median_homeprice,percent_change,cumulative_appreciation,city_ascii,state_name,county_name,latitude,longitude
0,2012,CA,416000.0,18.857143,18.857143,Los Angeles,California,Los Angeles,34.1141,-118.4068
1,2013,CA,499000.0,19.951923,42.571429,Los Angeles,California,Los Angeles,34.1141,-118.4068
2,2014,CA,520000.0,4.208417,48.571429,Los Angeles,California,Los Angeles,34.1141,-118.4068
3,2015,CA,558000.0,7.307692,59.428571,Los Angeles,California,Los Angeles,34.1141,-118.4068
4,2016,CA,585500.0,4.928315,67.285714,Los Angeles,California,Los Angeles,34.1141,-118.4068
...,...,...,...,...,...,...,...,...,...,...
368,2019,CA,289000.0,14.229249,92.666667,Sonora,California,Tuolumne,37.9819,-120.3828
369,2020,CA,318250.0,10.121107,112.166667,Sonora,California,Tuolumne,37.9819,-120.3828
370,2021,CA,400000.0,25.687353,166.666667,Sonora,California,Tuolumne,37.9819,-120.3828
371,2022,CA,417500.0,4.375000,178.333333,Sonora,California,Tuolumne,37.9819,-120.3828


In [3]:
# Data Cleaning for HPI dataframe

# Strip whitespace from "state" column for HPI
df_HPI_state['state'] = df_HPI_state['state'].str.strip()

# Drop null values from dataframe
df_HPI_state.dropna(inplace=True)

# Drop decimals in 'yr', 'year', and 'qtr' columns by converting to integer
df_HPI_state['yr'] = df_HPI_state['yr'].astype(int)
df_HPI_state['qtr'] = df_HPI_state['qtr'].astype(int)

# Set list of removal values for HPI
removal_yrs = [1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
               2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
               2010]

removal_qtrs = [1, 2, 3]

# Drop years outside of study period of 2012-2022 for HPI
df_HPI_state.drop(df_HPI_state[df_HPI_state['yr'].isin(removal_yrs)].index, inplace=True)

# Drop quarter 1, 2, and 3, rows and retain quarter 4 values for HPI
df_HPI_state.drop(df_HPI_state[df_HPI_state['qtr'].isin(removal_qtrs)].index, inplace=True)
df_HPI_state.reset_index(drop=True, inplace=True)

# Drop 'qtr' column and 'index_nsa' column for HPI
df_HPI_state.drop(columns=['qtr', 'index_nsa'], inplace=True)


# Convert 'yr' to datetime as 'year' and drop 'yr' column
df_HPI_state['year'] = pd.to_datetime(df_HPI_state['yr'], format='%Y').dt.year
df_HPI_state.drop(columns=['yr'], inplace=True)

# Reset index for HPI
df_HPI_state.reset_index(drop=True, inplace=True)

# Set the HPI dataframe index to 'year' and 'state'
df_HPI_state = df_HPI_state.set_index(['year','state'])

# Reset the HPI dataframe index
df_HPI_reset = df_HPI_state.reset_index()

# Checking dataframes for nulls
#df_HPI_state.isnull().sum()
#df_median_bystate.isnull().sum()
#df_median_bymetro.isnull().sum()
#df_latlong.isnull().sum()

In [4]:
# Data Cleaning for median home price by state and by metro dataframes

# Drop null values from dataframes
df_median_bystate.dropna(inplace=True)
df_median_bymetro.dropna(inplace=True)

# Drop decimals in 'year' column by converting to integer
df_median_bymetro['year'] = df_median_bymetro['year'].astype(int)

# Drop 'region_id' and 'size_rank' from median by metro dataframe
df_median_bymetro.drop(columns=['region_id', 'size_rank'], inplace=True)

# Rename region_name to city for median by metro dataframe
df_median_bymetro = df_median_bymetro.rename(columns={'region_name': 'city'})

# Trim state info from city name in 'region_name' column for median by metro dataframe
df_median_bymetro['city'] = df_median_bymetro['city'].str.replace(", CA", "")
df_median_bymetro['city'] = df_median_bymetro['city'].str.replace(", TX", "")
df_median_bymetro['city'] = df_median_bymetro['city'].str.replace(", FL", "")

# Calculate the percentage change for median home price from year-to-year
#df_HPI_reset['pct_appreciation'] = df_HPI_reset.groupby('state')['index_sa'].diff()
df_median_bymetro['percent_change'] = df_median_bymetro.groupby('city')['median_homeprice'].pct_change()
df_median_bymetro['cumulative_appreciation'] = (1 + df_median_bymetro['percent_change']).groupby(df_median_bymetro['city']).cumprod() - 1
df_median_bymetro['percent_change'] = df_median_bymetro['percent_change'] * 100
df_median_bymetro['cumulative_appreciation'] = df_median_bymetro['cumulative_appreciation'] * 100

# Drop nulls
df_median_bymetro.dropna(inplace=True)

In [5]:
# Data Cleaning for latitude and longitude dataframe

# Drop null values from dataframes
df_latlong.dropna(inplace=True)

# Remove unneeded columns from df_latlong
longlat_remove = ['county_fips', 'population', 'density', 
                  'source', 'military', 'incorporated', 
                  'timezone', 'zips', 'id', 'ranking']
df_latlong = df_latlong.drop(columns=longlat_remove)

# Rename 'state_id' to 'state'
df_latlong = df_latlong.rename(columns={'state_id': 'state'})


In [6]:
# Mergine latitude and longitude columns with median home price by metro dataframe

# Merge datasets on 'city' and 'state' columns
df_median_bymetro = pd.merge(df_median_bymetro, df_latlong, on=['city',"state"], how='left')

# Copy 'lat' and 'lng' values to new columns
df_median_bymetro['latitude'] = df_median_bymetro['lat']
df_median_bymetro['longitude'] = df_median_bymetro['lng']

# Drop 'lat' and 'lng'
df_median_bymetro.drop(['lat', 'lng'], axis=1, inplace=True)

In [7]:
# Calculate the percentage HPI growth from year-to-year
#df_HPI_reset['pct_appreciation'] = df_HPI_reset.groupby('state')['index_sa'].diff()
df_HPI_reset['percent_change'] = df_HPI_reset.groupby('state')['index_sa'].pct_change()
df_HPI_reset['cumulative_appreciation'] = (1 + df_HPI_reset['percent_change']).groupby(df_HPI_reset['state']).cumprod() - 1
df_HPI_reset['percent_change'] = df_HPI_reset['percent_change'] * 100
df_HPI_reset['cumulative_appreciation'] = df_HPI_reset['cumulative_appreciation'] * 100

#df_HPI_reset['cumulative_appreciation'] = df_HPI_reset['percent_change'].cumprod()

In [64]:
df_HPI_reset[df_HPI_reset['state'] == "CA"]

,year,state,index_sa,percent_change,cumulative_appreciation
49,2012,CA,169.25,11.399987,11.399987
50,2013,CA,199.99,18.162482,31.632989
51,2014,CA,214.93,7.470374,41.466465
52,2015,CA,230.47,7.230261,51.694859
53,2016,CA,246.41,6.916301,62.186533
54,2017,CA,266.16,8.015097,75.185941
55,2018,CA,279.25,4.918094,83.801751
56,2019,CA,291.85,4.512086,92.095044
57,2020,CA,326.22,11.776598,114.717304
58,2021,CA,390.19,19.609466,156.822221


In [9]:
# drop null values from dataframe
df_HPI_reset.dropna(inplace=True)

In [10]:
# Drop rows with 2011 in 'year' column
del_year = [2011]
df_HPI_reset.drop(df_HPI_reset[df_HPI_reset['year'].isin(del_year)].index, inplace=True)

In [11]:
# Plot the Percentage appreciation for each year, grouped by 'state'
df_HPI_reset.hvplot(
    x = "year",
    y = 'percent_change',
    xlabel = 'Year',
    ylabel = 'Percentage Appreciation',
    groupby = 'state',
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)

:DynamicMap   [state]
   :Curve   [year]   (percent_change)

In [12]:
# # Calculate the cumulative appreciation
# df_HPI_reset['appreciation_cumsum'] = df_HPI_reset.groupby('state')['pct_appreciation'].cumsum()
# df_HPI_reset.head()

In [13]:
# Plot Accumulated Percentage Appreciation grouped by 'state'
df_HPI_reset.hvplot(
    x = "year",
    y = 'cumulative_appreciation',
    xlabel = 'Year',
    ylabel = 'Cumulative Appreciation',
    groupby = 'state',
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)

:DynamicMap   [state]
   :Curve   [year]   (cumulative_appreciation)

In [14]:
# Create separate dataframes for CA, FL, and TX plotting of HPI
df_ca_hpi = df_HPI_reset[df_HPI_reset['state'] == 'CA']
df_fl_hpi = df_HPI_reset[df_HPI_reset['state'] == 'FL']
df_tx_hpi = df_HPI_reset[df_HPI_reset['state'] == 'TX']
df_us_hpi = df_HPI_reset[df_HPI_reset['state'] == 'USA']

In [15]:
# CA cumulative HPI plot
ca_hpi_plot_cum = df_ca_hpi.hvplot(
    x = "year",
    y = 'cumulative_appreciation',
    xlabel = 'Year',
    ylabel = 'Cumulative Percentage Appreciation',
    label = 'California Housing Price Appreciation from 2012 - 2022',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)
ca_hpi_plot_cum

:Curve   [year]   (cumulative_appreciation)

In [16]:
# FL cumulative HPI plot
fl_hpi_plot_cum = df_fl_hpi.hvplot(
    x = "year",
    y = 'cumulative_appreciation',
    xlabel = 'Year',
    ylabel = 'Cumulative Percentage Appreciation',
    label = 'Florida Housing Price Appreciation from 2012 - 2022',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)
fl_hpi_plot_cum

:Curve   [year]   (cumulative_appreciation)

In [17]:
# TX cumulative appreciation plot
tx_hpi_plot_cum = df_tx_hpi.hvplot(
    x = "year",
    y = 'cumulative_appreciation',
    xlabel = 'Year',
    ylabel = 'Cumulative Percentage Appreciation',
    label = 'Texas Housing Price Appreciation from 2012 - 2022',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)
tx_hpi_plot_cum

:Curve   [year]   (cumulative_appreciation)

In [18]:
# USA cumulative HPI plot
us_hpi_plot_cum = df_us_hpi.hvplot(
    x = "year",
    y = 'cumulative_appreciation',
    xlabel = 'Year',
    ylabel = 'Cumulative Percentage Appreciation',
    label = 'United States Housing Price Appreciation from 2012 - 2022',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)
us_hpi_plot_cum

:Curve   [year]   (cumulative_appreciation)

In [19]:
# Overlay CA, FL, and Texas cumulative appreciation
cum_hpi_overlay = ca_hpi_plot_cum * tx_hpi_plot_cum * fl_hpi_plot_cum * us_hpi_plot_cum
cum_hpi_overlay.opts(legend_position = 'top_left')

:Overlay
   .Curve.California_Housing_Price_Appreciation_from_2012_hyphen_minus_2022    :Curve   [year]   (cumulative_appreciation)
   .Curve.Texas_Housing_Price_Appreciation_from_2012_hyphen_minus_2022         :Curve   [year]   (cumulative_appreciation)
   .Curve.Florida_Housing_Price_Appreciation_from_2012_hyphen_minus_2022       :Curve   [year]   (cumulative_appreciation)
   .Curve.United_States_Housing_Price_Appreciation_from_2012_hyphen_minus_2022 :Curve   [year]   (cumulative_appreciation)

In [20]:
# CA year-to-year appreciation plot
ca_hpi_plot_y2y = df_ca_hpi.hvplot(
    x = "year",
    y = 'percent_change',
    xlabel = 'Year',
    ylabel = 'Year-to-Year Appreciation (%)',
    label = 'California Y2Y House Price Appreciation from 2012 - 2022 (%)',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)
ca_hpi_plot_y2y

:Curve   [year]   (percent_change)

In [21]:
# FL year-to-year appreciation plot
fl_hpi_plot_y2y = df_fl_hpi.hvplot(
    x = "year",
    y = 'percent_change',
    xlabel = 'Year',
    ylabel = 'Year-to-Year Appreciation (%)',
    label = 'Florida Y2Y House Price Appreciation from 2012 - 2022 (%)',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)
fl_hpi_plot_y2y

:Curve   [year]   (percent_change)

In [22]:
# FL year-to-year appreciation plot
tx_hpi_plot_y2y = df_tx_hpi.hvplot(
    x = "year",
    y = 'percent_change',
    xlabel = 'Year',
    ylabel = 'Year-to-Year Appreciation (%)',
    label = 'Texas Y2Y House Price Appreciation from 2012 - 2022 (%)',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)
tx_hpi_plot_y2y

:Curve   [year]   (percent_change)

In [23]:
# USA year-to-year appreciation plot
us_hpi_plot_y2y = df_us_hpi.hvplot(
    x = "year",
    y = 'percent_change',
    xlabel = 'Year',
    ylabel = 'Year-to-Year Appreciation (%)',
    label = 'United States Y2Y House Price Appreciation from 2012 - 2022 (%)',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)
us_hpi_plot_y2y

:Curve   [year]   (percent_change)

In [24]:
# Overlay CA, FL, and Texas year-to-year appreciation
y2y_hpi_overlay = ca_hpi_plot_y2y * tx_hpi_plot_y2y * fl_hpi_plot_y2y * us_hpi_plot_y2y
y2y_hpi_overlay.opts(legend_position = 'top_left', title='Y2Y House Price Index Appreciation 2012 - 2022')

:Overlay
   .Curve.California_Y2Y_House_Price_Appreciation_from_2012_hyphen_minus_2022_left_parenthesis_percent_right_parenthesis    :Curve   [year]   (percent_change)
   .Curve.Texas_Y2Y_House_Price_Appreciation_from_2012_hyphen_minus_2022_left_parenthesis_percent_right_parenthesis         :Curve   [year]   (percent_change)
   .Curve.Florida_Y2Y_House_Price_Appreciation_from_2012_hyphen_minus_2022_left_parenthesis_percent_right_parenthesis       :Curve   [year]   (percent_change)
   .Curve.United_States_Y2Y_House_Price_Appreciation_from_2012_hyphen_minus_2022_left_parenthesis_percent_right_parenthesis :Curve   [year]   (percent_change)

In [82]:
# Plot cumulative median home price appreciation for CA, FL, TX, and USA. 
ca_hpi_plot_cum = df_ca_hpi.hvplot(
    x = "year",
    y = 'cumulative_appreciation',
    xlabel = 'Year',
    ylabel = 'Median Home Price Appreciation (%)',
    label = 'CA Cumulative House Price Appreciation',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)

fl_hpi_plot_cum = df_fl_hpi.hvplot(
    x = "year",
    y = 'cumulative_appreciation',
    xlabel = 'Year',
    ylabel = 'YMedian Home Price Appreciation (%)',
    label = 'FL Cumulative House Price Appreciation',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)

tx_hpi_plot_cum = df_tx_hpi.hvplot(
    x = "year",
    y = 'cumulative_appreciation',
    xlabel = 'Year',
    ylabel = 'Median Home Price Appreciation (%)',
    label = 'TX Cumulative House Price Appreciation',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)

us_hpi_plot_cum = df_us_hpi.hvplot(
    x = "year",
    y = 'cumulative_appreciation',
    xlabel = 'Year',
    ylabel = 'Median Home Price Appreciation (%)',
    label = 'US Cumulative House Price Appreciation',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)


In [87]:
# Overlay CA, FL, TX, and USA cumulative median home price appreciation
cum_hpi_overlay = ca_hpi_plot_cum * fl_hpi_plot_cum * tx_hpi_plot_cum * us_hpi_plot_cum
cum_hpi_overlay.opts(legend_position = 'top_left', title='Cumulative Median House Price Appreciation 2012 - 2022')

:Overlay
   .Curve.CA_Cumulative_House_Price_Appreciation :Curve   [year]   (cumulative_appreciation)
   .Curve.FL_Cumulative_House_Price_Appreciation :Curve   [year]   (cumulative_appreciation)
   .Curve.TX_Cumulative_House_Price_Appreciation :Curve   [year]   (cumulative_appreciation)
   .Curve.US_Cumulative_House_Price_Appreciation :Curve   [year]   (cumulative_appreciation)

In [26]:
# Create separate dataframes for CA, FL, and TX plotting of HPI
df_ca_median_bystate = df_median_bystate[df_median_bystate['state_twoltr'] == 'CA']
df_fl_median_bystate = df_median_bystate[df_median_bystate['state_twoltr'] == 'FL']
df_tx_median_bystate = df_median_bystate[df_median_bystate['state_twoltr'] == 'TX']
#df_us_hpi = df_HPI_reset[df_HPI_reset['state'] == 'USA']

In [27]:
# Charting median home price by state for CA
ca_median_bystate_plot = df_ca_median_bystate.hvplot(
    x = "year",
    y = 'median_hp',
    xlabel = 'Year',
    ylabel = 'Median Home Price',
    label = 'California Median Home Price from 2012 - 2022',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)
ca_median_bystate_plot

:Curve   [year]   (median_hp)

In [28]:
# Charting median home price by state for FL
fl_median_bystate_plot = df_fl_median_bystate.hvplot(
    x = "year",
    y = 'median_hp',
    xlabel = 'Year',
    ylabel = 'Median Home Price',
    label = 'Florida Median Home Price from 2012 - 2022',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)
fl_median_bystate_plot

:Curve   [year]   (median_hp)

In [29]:
# Charting median home price by state for CA
tx_median_bystate_plot = df_tx_median_bystate.hvplot(
    x = "year",
    y = 'median_hp',
    xlabel = 'Year',
    ylabel = 'Median Home Price',
    label = 'Texas Median Home Price from 2012 - 2022',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)
tx_median_bystate_plot

:Curve   [year]   (median_hp)

In [30]:
# Overlay CA, FL, and Texas year-to-year appreciation
median_bystate_overlay = ca_median_bystate_plot * fl_median_bystate_plot * tx_median_bystate_plot
median_bystate_overlay.opts(yformatter='%.0f',legend_position = 'top_left', title='Median Home Price')

:Overlay
   .Curve.California_Median_Home_Price_from_2012_hyphen_minus_2022 :Curve   [year]   (median_hp)
   .Curve.Florida_Median_Home_Price_from_2012_hyphen_minus_2022    :Curve   [year]   (median_hp)
   .Curve.Texas_Median_Home_Price_from_2012_hyphen_minus_2022      :Curve   [year]   (median_hp)

In [31]:
# Create separate dataframes for CA, FL, and TX plotting of median home price by metro
df_ca_median_bymetro = df_median_bymetro[df_median_bymetro['state'] == 'CA']
df_fl_median_bymetro = df_median_bymetro[df_median_bymetro['state'] == 'FL']
df_tx_median_bymetro = df_median_bymetro[df_median_bymetro['state'] == 'TX']
#df_us_hpi = df_HPI_reset[df_HPI_reset['state'] == 'USA']

In [93]:
# Plot median home price by metro for CA
ca_map_plot = df_ca_median_bymetro.hvplot.points(
    'longitude', 
    'latitude', 
    geo=True, 
    size = 'median_homeprice',
    groupby = 'year',
    scale = .02,
    color='city',
    alpha=0.8,
    tiles='OSM',
    frame_width = 700,
    frame_height = 500
    )

ca_map_plot

:DynamicMap   [year]
   :Overlay
      .Tiles.I  :Tiles   [x,y]
      .Points.I :Points   [longitude,latitude]   (city,median_homeprice)

In [91]:
# Plot median home price by metro for FL
fl_map_plot = df_fl_median_bymetro.hvplot.points(
    'longitude', 
    'latitude', 
    geo=True, 
    size = 'median_homeprice',
    groupby = 'year',
    xlabel = 'longitude',
    ylabel = 'latitude',
    title = 'FL Median House Price by Metro Area and Year',
    scale = .02,
    color='city',
    alpha=0.8,
    tiles='OSM',
    frame_width = 700,
    frame_height = 500
    )

fl_map_plot

:DynamicMap   [year]
   :Overlay
      .Tiles.I  :Tiles   [x,y]
      .Points.I :Points   [longitude,latitude]   (city,median_homeprice)

In [67]:
# Plot median home price by metro for TX
tx_map_plot = df_tx_median_bymetro.hvplot.points(
    'longitude', 
    'latitude', 
    geo=True, 
    size = 'median_homeprice',
    groupby = 'year',
    scale = .02,
    color='city',
    alpha=0.8,
    tiles='OSM',
    frame_width = 700,
    frame_height = 500
    )

tx_map_plot

:DynamicMap   [year]
   :Overlay
      .Tiles.I  :Tiles   [x,y]
      .Points.I :Points   [longitude,latitude]   (city,median_homeprice)

In [63]:
# Overlay CA, FL, and Texas year-to-year appreciation
median_bymetro_overlay = ca_map_plot * fl_map_plot #* tx_map_plot
median_bymetro_overlay#.opts(yformatter='%.0f',legend_position = 'top_left', title='Median Home Price for metro areas by Year')

:DynamicMap   [year]
   :Overlay
      .Tiles.I   :Tiles   [x,y]
      .Points.I  :Points   [longitude,latitude]   (city,median_homeprice)
      .Tiles.II  :Tiles   [x,y]
      .Points.II :Points   [longitude,latitude]   (city,median_homeprice)

In [48]:
df_median_bymetro.reset_index(drop=True, inplace=True)

In [56]:
#df_ca_median_bymetro_top3 = df_ca_median_bymetro.sortby
df_ca_median_bymetro['sum_cumulative'] = df_ca_median_bymetro.groupby('city')['cumulative_appreciation'].transform('sum')
#sorted_groups = df_ca_median_bymetro.groupby('city').apply(lambda x: x.sort_values(by='cumulative_appre'))

/var/folders/h4/gc4497k550l91t9cq44cscm00000gn/T/ipykernel_19684/3699144337.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ca_median_bymetro['sum_cumulative'] = df_ca_median_bymetro.groupby('city')['cumulative_appreciation'].transform('sum')


In [38]:
# Charting cumulative percentage change in median home price for CA, FL, and TX
ca_median_bymetro_cumplot = df_ca_median_bymetro.hvplot(
    x = "year",
    y = 'cumulative_appreciation',
    xlabel = 'Year',
    ylabel = 'Cumulative Percentage Appreciation',
    label = 'California Cumulative Median Home Price Appreciation',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)

fl_median_bymetro_cumplot = df_fl_median_bymetro.hvplot(
    x = "year",
    y = 'cumulative_appreciation',
    xlabel = 'Year',
    ylabel = 'Cumulative Percentage Appreciation',
    label = 'Florida Cumulative Median Home Price Appreciation',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)

tx_median_bymetro_cumplot = df_tx_median_bymetro.hvplot(
    x = "year",
    y = 'cumulative_appreciation',
    xlabel = 'Year',
    ylabel = 'Cumulative Percentage Appreciation',
    label = 'Texas Cumulative Median Home Price Appreciation',
    width = 800,
    height = 500
    #title = 'Sale Price Per Square Foot and Average Gross Rent - 2010-2016 - By Neighborhood',
)

In [39]:
# Overlay CA, FL, and Texas cumulative median home price appreciation
median_bymetro_cumulative_overlay = ca_median_bymetro_cumplot * fl_median_bymetro_cumplot * tx_median_bymetro_cumplot * us_hpi_plot_y2y
median_bymetro_cumulative_overlay.opts(legend_position = 'top_left', title='Y2Y House Price Index Appreciation 2012 - 2022')

:Overlay
   .Curve.California_Cumulative_Median_Home_Price_Appreciation                                                              :Curve   [year]   (cumulative_appreciation)
   .Curve.Florida_Cumulative_Median_Home_Price_Appreciation                                                                 :Curve   [year]   (cumulative_appreciation)
   .Curve.Texas_Cumulative_Median_Home_Price_Appreciation                                                                   :Curve   [year]   (cumulative_appreciation)
   .Curve.United_States_Y2Y_House_Price_Appreciation_from_2012_hyphen_minus_2022_left_parenthesis_percent_right_parenthesis :Curve   [year]   (percent_change)

In [40]:
ca_median_bystate_plot

:Curve   [year]   (median_hp)

In [61]:
df_median_bystate[df_median_bystate['state_twoltr'] == 'CA']

,year,state,state_twoltr,median_hp
0,2011,California,CA,274662.3710
1,2012,California,CA,295252.6018
2,2013,California,CA,361030.1213
3,2014,California,CA,392947.2814
4,2015,California,CA,413380.8063
5,2016,California,CA,439949.5584
6,2017,California,CA,486925.3254
7,2018,California,CA,529729.2768
8,2019,California,CA,538160.8160
9,2020,California,CA,586704.5464


In [42]:
# Create separate dataframes for CA, FL, and TX plotting of HPI
df_ca_median_bymetro = df_median_bymetro[df_median_bymetro['state'] == 'CA']
df_fl_median_bymetro = df_median_bymetro[df_median_bymetro['state'] == 'FL']
df_tx_median_bymetro = df_median_bymetro[df_median_bymetro['state'] == 'TX']
#df_us_hpi = df_HPI_reset[df_HPI_reset['state'] == 'USA']

In [43]:
df_median_bymetro.groupby('year')

In [44]:
# Plot the data using hvPlot and Geoviews
map_plot = df_ca_median_bymetro.hvplot.points(
    'longitude', 
    'latitude', 
    geo=True, 
    size = 'median_homeprice',
    scale = .02,
    color='city',
    alpha=0.8,
    tiles='OSM',
    frame_width = 700,
    frame_height = 500
    )

map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (city,median_homeprice)

In [92]:
fl_map_plot

:DynamicMap   [year]
   :Overlay
      .Tiles.I  :Tiles   [x,y]
      .Points.I :Points   [longitude,latitude]   (city,median_homeprice)